In [ ]:
import sys
import os

# Add the folder paths to sys.path
sys.path.append(os.path.abspath('/scratch/user/s4647285/DeepHis2Exp/Models/BLEEP'))
sys.path.append(os.path.abspath('/scratch/user/s4647285/DeepHis2Exp/Dataloader'))
sys.path.append(os.path.abspath("/scratch/user/s4647285/DeepHis2Exp/Wilson/BLEEP_Experiment"))

import torch
import pytorch_lightning as pl
import bleep as bl
import scanpy as sc
import stlearn as st
import pandas as pd
import BLEEP_pfizer_dataloader as bpd


In [ ]:
INPUT_PATH = "/QRISdata/Q1851/Wilson/Pfizer/PROCESSED_DATA/" 

In [ ]:
# Load the gene list
file_path = "/QRISdata/Q1851/Wilson/Pfizer/PROCESSED_DATA/samples_markers_intersection.csv"
gene_list_df = pd.read_csv(file_path, header = None)
gene_list = gene_list_df.to_numpy().flatten()
print("The gene list is loaded")

#load sample list 
vis_sample_list = ["VLP78_A",  "VLP78_D",  "VLP79_A",  "VLP79_D",  "VLP80_A",  "VLP80_D",  "VLP81_A",  "VLP82_A",  "VLP82_D",  "VLP83_A",  "VLP83_D"]

batch_size = 1

dataloaders = bpd.BLEEP_Pfizer_data_loader(sample_list = vis_sample_list, batch_size = batch_size, shuffle = True, desired_genes = gene_list, subset_size = 10)

train_dataloader = dataloaders[0]
test_dataloader = dataloaders[1]
val_dataloader = dataloaders[2]
# specify number of epochs
print("The data is loaded")


In [ ]:
num_epoch = 150
print("The model has the maximum of", num_epoch, "epochs")

In [ ]:
#create an instance of the model
model = bl.BLEEP(n_genes = len(gene_list))

#train
trainer = pl.Trainer(max_epochs=num_epoch, check_val_every_n_epoch=5, default_root_dir = "/scratch/user/s4647285/DeepHis2Exp/Wilson/BLEEP_Experiment/BLEEP_Model_1_rerun.ckpt")
trainer.fit(model, train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)


In [ ]:
train_loss_np = []

for loss in model.train_loss:
    train_loss_np.append(loss.cpu().detach().numpy())
    
import matplotlib.pyplot as plt

plt.plot(train_loss_np)
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
val_loss_np = []

for loss in model.val_loss:
    val_loss_np.append(loss.cpu().detach().numpy())
    
import matplotlib.pyplot as plt

plt.plot(val_loss_np)
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
#test
trainer.test(model, test_dataloader)